Mount Google Drive (optional)

In [33]:
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# # os.chdir("/content/drive/MyDrive/....")  # file path
# print(os.getcwd())

# **HW2 : Decision Tree and Random Forest**
In *assignment 2*, you need to finish :

1. Basic Part : Implement a **Decision Tree** model and predict whether the patients in the validation set have diabetes
> * Step 1 : Load the input data
> * Step 2 : Calculate the Entropy and Information Gain
> * Step 3 : Find the Best Split
> * Step 4 : Split into 2 branches
> * Step 5 : Build decision tree
> * Step 6 : Save the answers from step2 to step5
> * Step 7 : Split data into training set and validation set
> * Step 8 : Train a decision tree model with training set
> * Step 9 : Predict the cases in the *validation set* by using the model trained in *Step8*
> * Step 10 : Calculate the f1-score of your predictions in *Step9*
> * Step 11 : Write the Output File

2. Advanced Part : Build a **Random Forest** model to make predictions
> * Step 1 : Load the input data
> * Step 2 : Load the test data
> * Step 3 : Build a random forest
> * Step 4 : Predict the cases in the test data by using the model trained in *Step3*
> * Step 5 : Save the predictions(from *Step 4*) in a csv file



# **Basic Part** (60%)
In this part, your need to implement a Decision Tree model by completing the following given functions.

Also, you need to run these functions with the given input variables and save the output in a csv file **hw2_basic.csv**

## Import Packages


> Note : You **cannot** import any other packages in both basic part and advanced part






In [34]:
import numpy as np
import pandas as pd
import math
import random
from numpy import sqrt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

## Step1: Load the input data
First, load the input file **hw2_input_basic.csv**

In [35]:
input_data = pd.read_csv('hw2_input_basic.csv')
input_data

,age,bmi,gender,height,weight,glucose_apache,heart_rate_apache,resprate_apache,sodium_apache,diabetes_mellitus
0,70.0,25.984659,1,172.7,77.50,116.0,101.0,49.0,137.0,0
1,30.0,31.310368,1,170.2,90.70,71.0,39.0,33.0,144.0,0
2,54.0,24.388824,1,177.8,77.10,120.0,120.0,31.0,141.0,0
3,65.0,34.141074,0,170.2,98.90,73.0,48.0,36.0,140.0,1
4,49.0,22.564743,1,172.7,67.30,207.0,119.0,6.0,144.0,0
5,62.0,29.424010,0,154.9,70.60,113.0,60.0,32.0,137.0,0
6,85.0,27.673574,1,154.9,66.40,102.0,49.0,36.0,142.0,0
7,65.0,22.269432,1,177.8,70.40,333.0,59.0,6.0,145.0,1
8,85.0,35.879362,0,165.1,97.80,124.0,92.0,30.0,136.0,0
9,81.0,20.859375,0,160.0,53.40,136.0,118.0,52.0,138.0,0


## Global attributes
Define the global attributes
> Note : You **cannot** modify the values of these attributes we given in the basic part

In [36]:
max_depth = 2
depth = 0
min_samples_split = 2
n_features = input_data.shape[1] - 1

> You can add your own global attributes here

## Step2 : Calculate the Entropy and Information Gain 
Calculate the information gain and entropy values before separate data into left subtree and right subtree

In [37]:
def entropy(data):
  """
  This function measures the amount of uncertainty in a probability distribution
  args: 
  * data(type: DataFrame): the data you're calculating for the entropy
  return:
  * entropy_value(type: float): the data's entropy
  """
  
  total = 0
  size = len(data)
  if(size == 0):
    return 0
  for i in range(size):
    if(data.loc[i, "diabetes_mellitus"] == 1):
      total += 1
  rate = total/size
  if(rate == 0 or rate == 1):
    return 0 
  entropy_value = -rate * math.log2(rate) - (1-rate) * math.log2(1-rate)
  return entropy_value

# [Note] You have to save the value of "ans_entropy" into the output file
ans_entropy = entropy(input_data)
print("ans_entropy = ", ans_entropy)

ans_entropy =  0.9871377743721863


In [38]:
def information_gain(data, mask):
  """
  This function will calculate the information gain
  args:
  * data(type: DataFrame): the data you're calculating for the information gain
  * mask(type: Series): partition information(left/right) of current input data, 
    - boolean 1(True) represents split to left subtree
    - boolean 0(False) represents split to right subtree
  return:
  * ig(type: float): the information gain you can obtain by classify data with this given mask
  """
  
  left = pd.DataFrame()
  right = pd.DataFrame()
  for i in range(data.shape[0]):
    if(mask[i] == True):
      left = pd.concat([left, data.iloc[[i]]], ignore_index=True)
    else:
      right = pd.concat([right, data.iloc[[i]]], ignore_index=True)
 
  
  ig = entropy(data) - len(left)/(len(left) + len(right)) * entropy(left) - len(right)/(len(left) + len(right)) * entropy(right)
  
  return ig
  
# [Note] You have to save the value of "ans_informationGain" into your output file
temp1 = np.zeros((int(input_data.shape[0]/4), 1), dtype=bool)
temp2 = np.ones(((input_data.shape[0]-int(input_data.shape[0]/4), 1)), dtype=bool)
temp_mask = np.concatenate((temp1, temp2))
df_mask = pd.DataFrame(temp_mask, columns=['mask'])
ans_informationGain = information_gain(input_data, df_mask['mask'])
print("ans_informationGain = ", ans_informationGain)




ans_informationGain =  0.08345988684807157


## Step3 : Find the Best Split
Find the best split combination, **feature** and **threshold**, by calculating the information gain


In [39]:
def find_best_split(data):
  """
  This function will find the best split combination of data
  args:
  * data(type: DataFrame): the input data
  return
  * best_ig(type: float): the best information gain you obtain
  * best_threshold(type: float): the value that splits data into 2 branches
  * best_feature(type: string): the feature that splits data into 2 branches
  """
  temp = pd.DataFrame()
  ig = 0
  best_threshold = 0
  best_feature = 0
  threshold = 0
  all_true = sum(data["diabetes_mellitus"])
  ori_ig = entropy(data)
  for i in range(len(data.columns)-1):
        
    if(~data[data.columns[i]].isin([0,1]).all()):
      left = 0
      temp = data[[data.columns[i] , "diabetes_mellitus" ]]
      temp1 = temp.sort_values(temp.columns[0], ascending = True)
      
      temp1.reset_index(drop=True, inplace= True)
      for j in range(1,len(data)):
        if(temp1[data.columns[i]][j] != temp1[data.columns[i]][j-1]):
          threshold = (temp1[data.columns[i]][j] + temp1[data.columns[i]][j-1] ) / 2
          temp2 = np.ones((j, 1), dtype=bool)
          temp3 = np.zeros((len(data) - j, 1), dtype=bool)
          temp_mask = np.concatenate((temp2, temp3))
          df_mask = pd.DataFrame(temp_mask, columns=['mask'])
          # t = information_gain(temp1, df_mask['mask'])
          left += temp1["diabetes_mellitus"].iat[j-1]
          right = all_true - left
          fa_en = lambda rate: 0 if (rate == 1 or rate == 0) else -rate * math.log2(rate) - (1-rate) * math.log2(1-rate)
          t = ori_ig - (j/temp1.shape[0]) * fa_en(left/j) - ((temp1.shape[0]-j)/temp1.shape[0]) * fa_en(right/(temp1.shape[0]-j))
          if(t > ig):
            ig = t
            best_threshold = threshold
            best_feature = data.columns[i]
        else:
            left += temp1["diabetes_mellitus"].iat[j-1]
            continue
    
    else:
      temp = data[[data.columns[i] , "diabetes_mellitus" ]]
      temp1 = temp.sort_values(temp.columns[0], ascending = True)
      temp1.reset_index(drop=True, inplace= True)
      
      t = information_gain(temp1, temp1[data.columns[i]])
      if(t > ig):
        ig = t
        best_threshold = threshold
        best_feature = data.columns[i]
  best_ig = ig
  return best_ig, best_threshold, best_feature


# [Note] You have to save the value of "ans_ig", "ans_value", and "ans_name" into the output file
ans_ig, ans_value, ans_name = find_best_split(input_data)


print("ans_ig = ", ans_ig)
print("ans_value = ", ans_value)
print("ans_name = ", ans_name)


ans_ig =  0.3522950515812332
ans_value =  235.5
ans_name =  glucose_apache


## Step4 : Split into 2 branches
Using the best split combination you find in function *find_best_split()* to split data into Left Subtree and Right Subtree 

In [40]:
def make_partition(data, feature, threshold):
  """
  This function will split the data into 2 branches
  args:
  * data(type: DataFrame): the input data
  * feature(type: string): the attribute(column name)
  * threshold(type: float): the threshold for splitting the data
  return:
  * left(type: DataFrame): the divided data that matches(less than or equal to) the assigned feature's threshold
  * right(type: DataFrame): the divided data that doesn't match the assigned feature's threshold
  """
  
  left = pd.DataFrame()
  right = pd.DataFrame()
  
  for i in range(data.shape[0]):
    
    if(data[feature].iloc[i] <= threshold):
      left = pd.concat([left, data.iloc[[i]]], ignore_index=True)
    else:
      right = pd.concat([right, data.iloc[[i]]], ignore_index=True)
      
  return left, right


# [Note] You have to save the value of "ans_left" into the output file
left, right = make_partition(input_data, 'age', 61.0)
ans_left = left.shape[0]
print("ans_left = ", ans_left)

ans_left =  10


## Step5 : Build Decision Tree
Use the above functions to implement the decision tree

Instructions: 
1.  If current depth < max_depth and the remaining number of samples > min_samples_split: continue to classify those samples
2.  Use function *find_best_split()* to find the best split combination
3.  If the obtained information gain is **greater than 0**: can build a deeper decision tree (add depth)
4. Use function *make_partition()* to split the data into two parts
5. Save the features and corresponding thresholds (starting from the root) used by the decision tree into *ans_features[]* and *ans_thresholds[]* respectively




In [41]:
def build_tree(data, max_depth, min_samples_split, depth):
  """
  This function will build the decision tree
  args:
  * data(type: DataFrame): the data you want to apply to the decision tree
  * max_depth: the maximum depth of a decision tree
  * min_samples_split: the minimum number of instances required to do partition
  * depth: the height of the current decision tree
  return:
  * subtree: the decision tree structure including root, branch, and leaf (with the attributes and thresholds)
  """

  # check the condition of current depth and the remaining number of samples
  if depth < max_depth and len(data) > min_samples_split:
    # call find_best_split() to find the best combination
    ans_ig, threshold, feature = find_best_split(data)
    # check the value of information gain is greater than 0 or not 
    if ans_ig > 0 :
      # update the depth
      depth += 1
      # call make_partition() to split the data into two parts
      left_tree, right_tree = make_partition(data, feature, threshold)
      # If there is no data split to the left tree OR no data split to the left tree
      if left_tree.shape[0] == 0 or right_tree.shape[0] == 0:
        # return the label of the majority
        if(sum(data["diabetes_mellitus" ]) >= len(data)/2):
          label = 1
        else:
          label = 0
        return label
      else:
        question = "{} {} {}".format(feature, "<=", threshold)
        subtree = {question: []}
        ans_features.append(feature)
        ans_thresholds.append(threshold)
        # call function build_tree() to recursively build the left subtree and right subtree
        left_subtree = build_tree(left_tree, max_depth, min_samples_split, depth)
        right_subtree = build_tree(right_tree, max_depth, min_samples_split, depth)
        if left_subtree == right_subtree:
          subtree = left_subtree
        else:
          subtree[question].append(left_subtree)
          subtree[question].append(right_subtree)
    else:
      # return the label of the majority
      if(sum(data["diabetes_mellitus" ]) >= len(data)/2):
          label = 1
      else:
          label = 0
      return label
  else:
    # return the label of the majority
    if(sum(data["diabetes_mellitus" ]) >= len(data)/2):
      label = 1
    else:
      label = 0
    return label

  return subtree

An example of the output from *build_tree()* 
```
{'bmi <= 33.5': [1, {'age <= 68.5': [0, 1]}]}
```
Therefore, 
```
ans_features = ['bmi', 'age']
ans_thresholds = [33.5, 68.5]
```



In [42]:
ans_features = []
ans_thresholds = []

decisionTree = build_tree(input_data, max_depth, min_samples_split, depth)
decisionTree

{'glucose_apache <= 235.5': [{'heart_rate_apache <= 143.5': [0, 1]}, 1]}

In [43]:
# [Note] You have to save the features in the "decisionTree" structure (from root to branch and leaf) into the output file
ans_features

['glucose_apache', 'heart_rate_apache']

In [44]:
# [Note] You have to save the corresponding thresholds for the features in the "ans_features" list into the output file
ans_thresholds

[235.5, 143.5]

## Step6 : Save answers

In [45]:
basic = []
basic.append(ans_entropy)
basic.append(ans_informationGain)
basic.append(ans_ig)
basic.append(ans_value)
basic.append(ans_name)
basic.append(ans_left)
for i in range(len(ans_features)):
  basic.append(ans_features[i])
for m in range(len(ans_thresholds)):
  basic.append(ans_thresholds[m])

## Step7 : Split data
Split data into training set and validation set
> Note: We have split the data into training set and validation. You **cannot** change the distribution of the data.

In [46]:
num_train = 20
num_validation = 10

training_data = input_data.iloc[:num_train]
validation_data = input_data.iloc[-num_validation:]

y_train = training_data[["diabetes_mellitus"]]
x_train = training_data.drop(['diabetes_mellitus'], axis=1)
y_validation = validation_data[["diabetes_mellitus"]]
x_validation = validation_data.drop(['diabetes_mellitus'], axis=1)
y_validation = y_validation.values.flatten()

print(input_data.shape)
print(training_data.shape)
print(validation_data.shape)

(30, 10)
(20, 10)
(10, 10)


## Step8 to Step10 : Make predictions with a decision tree

Define the attributions of the decision tree
> You **cannot** modify the values of these attributes in this part

In [47]:
max_depth = 2
depth = 0
n_features = x_train.shape[1]

We have finished the function '*classify_data()*' below, however, you can modify this function if you prefer completing it on your own way.

In [48]:
def classify_data(instance, tree):
  """
  This function will predict/classify the input instance
  args:
  * instance: a instance(case) to be predicted
  return:
  * answer: the prediction result (the classification result)
  """
  
  equation = list(tree.keys())[0] 
  if equation.split()[1] == '<=':
    temp_feature = equation.split()[0]
    temp_threshold = equation.split()[2]
    if instance[temp_feature] > float(temp_threshold):
      answer = tree[equation][1]
    else:
      answer = tree[equation][0]
  else:
    if instance[equation.split()[0]] in (equation.split()[2]):
      answer = tree[equation][0]
    else:
      answer = tree[equation][1]

  if not isinstance(answer, dict):
    return answer
  else:
    return classify_data(instance, answer)


def make_prediction(tree, data):
  """
  This function will use your pre-trained decision tree to predict the labels of all instances in data
  args:
  * tree: the decision tree
  * data: the data to predict
  return:
  * y_prediction: the predictions
  """
  y_prediction = []
  for i in range(data.shape[0]):
    y_prediction.append(classify_data(data.iloc[i], tree))
  # [Note] You can call the function classify_data() to predict the label of each instance


  return y_prediction


def calculate_score(y_true, y_pred):
  """
  This function will calculate the f1-score of the predictions
  args:
  * y_true: the ground truth
  * y_pred: the predictions
  return:
  * score: the f1-score
  """
  score = f1_score(y_true,y_pred)
  
  return score

In [49]:
decision_tree = build_tree(training_data, max_depth, min_samples_split, depth)

y_pred = make_prediction(decision_tree, x_validation)

# [Note] You have to save the value of "ans_f1score" the your output file
ans_f1score = calculate_score(y_validation, y_pred)
print("ans_f1score = ", ans_f1score)

ans_f1score =  0.6666666666666666


## Step11 : Write the Output File
Save all of your answers in a csv file, named as **hw2_basic.csv**

In [50]:
ans_path = 'hw2_basic.csv'

# [Note] You have to save the value of "ans_f1score" into the output file
basic.append(ans_f1score)
print(basic)

pd.DataFrame(basic).to_csv(ans_path, header = None, index = None)

[0.9871377743721863, 0.08345988684807157, 0.3522950515812332, 235.5, 'glucose_apache', 10, 'glucose_apache', 'heart_rate_apache', 235.5, 143.5, 0.6666666666666666]


# **Advanced Part** (35%)

## Step1: Load the input data
First, load the input file **hw2_input_advanced.csv**

In [51]:
advanced_data = pd.read_csv('hw2_input_advanced.csv')
print(advanced_data.shape[0])

8379


You can split *advanced_data* into training set and validaiton set

In [52]:
data_size = advanced_data.shape[0]
num_train = 6000
num_validation = data_size - num_train

training_data = advanced_data.iloc[:num_train]
validation_data = advanced_data.iloc[-num_validation:]

y_train = training_data[["diabetes_mellitus"]]
x_train = training_data.drop(['diabetes_mellitus'], axis=1)
y_validation = validation_data[["diabetes_mellitus"]]
x_validation = validation_data.drop(['diabetes_mellitus'], axis=1)
y_validation = y_validation.values.flatten()

print(advanced_data.shape)
print(training_data.shape)
print(validation_data.shape)



(8379, 25)
(6000, 25)
(2379, 25)


## Step2 : Load the test data
Load the input file **hw2_input_test.csv** to make predictions with the pre-trained random forest model

In [53]:
x_test = pd.read_csv('hw2_input_test.csv')
x_test

,age,bmi,gender,height,weight,arf_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,...,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,apache_4a_hospital_death_prob,apache_4a_icu_death_prob
0,62,32.866392,1,177.80,103.9,1,31.0,10.30,4,6,...,36.4,0,157,26,134,36.1,0,4.56,0.06,0.03
1,82,23.582766,0,157.50,58.5,0,26.0,0.54,3,4,...,32.8,0,42,25,142,36.1,0,6.00,0.14,0.06
2,61,31.684520,1,172.70,94.5,0,16.0,1.11,4,6,...,35.3,0,129,6,131,36.8,0,8.59,0.05,0.03
3,58,45.156250,0,160.00,115.6,0,19.0,0.70,1,4,...,30.1,1,131,23,138,34.9,1,16.03,0.33,0.22
4,74,25.817016,1,172.70,77.0,0,25.0,0.93,4,6,...,34.5,0,55,12,135,36.3,0,45.80,0.12,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,73,17.943584,0,157.48,44.5,0,12.0,0.30,4,6,...,33.8,0,129,9,144,36.9,0,7.70,0.02,0.01
836,79,29.049732,1,167.60,81.6,0,48.0,2.19,4,6,...,42.7,0,163,9,139,36.4,0,10.77,0.06,0.03
837,85,24.627827,0,152.40,57.2,0,11.0,0.48,3,5,...,29.5,0,67,9,139,36.6,0,7.35,0.16,0.05
838,68,32.510940,1,193.00,121.1,0,14.0,0.64,4,6,...,37.5,0,61,10,140,36.9,1,16.02,0.00,0.00


## Step3 : Build a Random Forest

Define the attributions of the random forest
> * You **can** modify the values of these attributes in advanced part
> * Each tree can have different attribute values
> * There must be **at least** 3 decision trees in the random forest model
> * Must use function *build_tree()* to build a random forest model
> * These are the parameters you can adjust : 


    ```
    max_depth = 
    depth = 0
    min_samples_split = 
    
    # total number of trees in a random forest
    n_trees = 

    # number of features to train a decision tree
    n_features = 

    # the ratio to select the number of instances
    sample_size = 
    n_samples = int(training_data.shape[0] * sample_size)
    ```




In [54]:
# Define the attributes

max_depth = 5
depth = 0
min_samples_split = 10

# total number of trees in a random forest
n_trees = 550

# number of features to train a decision tree
n_features = 13

# the ratio to select the number of instances
sample_size = 0.1
n_samples = int(training_data.shape[0] * sample_size)
'''def build_tree(data, max_depth, min_samples_split, depth):'''

'def build_tree(data, max_depth, min_samples_split, depth):'

In [55]:



def build_forest(data, n_trees, n_features, n_samples):
  a = [*range(24)]
  """
  This function will build a random forest.
  args:
  * data: all data that can be used to train a random forest
  * n_trees: total number of tree
  * n_features: number of features
  * n_samples: number of instances
  return:
  * forest: a random forest with 'n_trees' of decision tree
  """
  
  forest = []
  for i in range(n_trees):
    new_data = pd.DataFrame()
    new_data = data.sample(n=n_samples)
    new_data_tree = pd.DataFrame()
    b = random.sample(a,k=n_features)
    for j in range(len(b)):
      new_data_tree[data.columns[b[j]]] = new_data[data.columns[b[j]]]
    new_data_tree["diabetes_mellitus"] = new_data["diabetes_mellitus"]
    new_data_tree = new_data_tree.reset_index(drop=True)
    
    tree = build_tree(new_data_tree, max_depth, min_samples_split, depth)
    if(tree != 0 and tree != 1):
      forest.append(tree)
      print(tree)
  return forest


In [61]:

forest = build_forest(training_data, n_trees, n_features, n_samples)
forest


{'glucose_apache <= 164.5': [{'glucose_apache <= 85.5': [{'hematocrit_apache <= 40.7': [{'resprate_apache <= 15.5': [1, {'hematocrit_apache <= 38.349999999999994': [0, 1]}]}, 0]}, {'creatinine_apache <= 1.9649999999999999': [0, {'resprate_apache <= 10.5': [1, {'hematocrit_apache <= 24.35': [1, 0]}]}]}]}, {'resprate_apache <= 30.5': [{'glucose_apache <= 222.5': [{'bmi <= 31.872313034999998': [{'creatinine_apache <= 0.63': [1, 0]}, {'glucose_apache <= 219.0': [1, 0]}]}, {'height <= 191.79500000000002': [{'apache_4a_icu_death_prob <= 0.76': [1, 0]}, 0]}]}, {'glucose_apache <= 249.5': [{'bmi <= 52.78616082': [0, 1]}, {'apache_4a_hospital_death_prob <= 0.865': [1, 0]}]}]}]}
{'glucose_apache <= 169.5': [{'glucose_apache <= 82.5': [{'bmi <= 27.938894565': [{'wbc_apache <= 34.05': [{'age <= 77.0': [0, 1]}, 1]}, {'age <= 52.5': [0, 1]}]}, {'bmi <= 26.619904575': [0, {'age <= 46.5': [{'apache_4a_hospital_death_prob <= 0.20500000000000002': [0, 1]}, 0]}]}]}, {'sodium_apache <= 129.5': [{'age <= 6

{'resprate_apache <= 39.5': [{'bmi <= 23.838643599999997': [{'creatinine_apache <= 1.335': [{'apache_4a_hospital_death_prob <= 0.055': [0, {'weight <= 75.8': [0, 1]}]}, {'sodium_apache <= 129.5': [1, {'wbc_apache <= 16.950000000000003': [1, 0]}]}]}, {'creatinine_apache <= 0.505': [1, {'resprate_apache <= 15.5': [{'bmi <= 55.558959650000006': [1, 0]}, {'creatinine_apache <= 1.4649999999999999': [0, 1]}]}]}]}, {'apache_4a_hospital_death_prob <= 0.355': [{'weight <= 79.95': [{'wbc_apache <= 31.8': [0, 1]}, {'map_apache <= 161.5': [0, 1]}]}, {'bmi <= 26.48886173': [1, {'apache_4a_hospital_death_prob <= 0.43': [1, 0]}]}]}]}
{'glucose_apache <= 164.5': [{'creatinine_apache <= 2.75': [{'heart_rate_apache <= 110.5': [{'bun_apache <= 10.5': [{'creatinine_apache <= 0.5349999999999999': [1, 0]}, 0]}, {'glucose_apache <= 95.5': [{'height <= 157.49': [1, 0]}, 0]}]}, {'resprate_apache <= 9.5': [1, {'heart_rate_apache <= 139.0': [{'resprate_apache <= 44.0': [1, 0]}, 0]}]}]}, {'resprate_apache <= 27.5

{'bun_apache <= 26.5': [{'creatinine_apache <= 0.785': [{'wbc_apache <= 6.25': [{'gcs_verbal_apache <= 1.5': [1, 0]}, {'heart_rate_apache <= 46.5': [0, {'height <= 185.3': [1, 0]}]}]}, {'height <= 182.89': [{'height <= 177.9': [0, 1]}, {'height <= 182.95': [0, {'bun_apache <= 21.5': [0, 1]}]}]}]}, {'heart_rate_apache <= 94.5': [{'height <= 184.15': [{'wbc_apache <= 8.350000000000001': [{'creatinine_apache <= 3.715': [1, 0]}, 1]}, 0]}, {'height <= 161.0': [{'apache_4a_hospital_death_prob <= -0.485': [0, {'gcs_eyes_apache <= 3.5': [0, 1]}]}, {'apache_4a_hospital_death_prob <= 0.015': [1, 0]}]}]}]}
{'glucose_apache <= 172.5': [{'bmi <= 29.23715503': [{'glucose_apache <= 85.5': [{'weight <= 50.45': [0, {'hematocrit_apache <= 24.200000000000003': [1, 0]}]}, {'height <= 170.19': [{'weight <= 73.75': [0, 1]}, 0]}]}, {'resprate_apache <= 34.5': [{'bmi <= 44.404500215': [{'apache_4a_icu_death_prob <= 0.005': [0, 1]}, 0]}, {'glucose_apache <= 75.5': [1, 0]}]}]}, {'resprate_apache <= 29.5': [{'cr

{'glucose_apache <= 192.5': [{'resprate_apache <= 29.5': [{'glucose_apache <= 156.5': [{'hematocrit_apache <= 41.55': [{'glucose_apache <= 62.0': [1, 0]}, 0]}, 1]}, {'resprate_apache <= 55.5': [{'hematocrit_apache <= 24.700000000000003': [{'hematocrit_apache <= 21.85': [0, 1]}, 0]}, 0]}]}, {'resprate_apache <= 38.5': [{'creatinine_apache <= 1.14': [{'height <= 175.25': [1, {'hematocrit_apache <= 34.8': [0, 1]}]}, {'weight <= 166.5': [{'weight <= 60.66': [0, 1]}, 0]}]}, {'weight <= 100.80000000000001': [{'map_apache <= 157.0': [1, 0]}, 0]}]}]}
{'creatinine_apache <= 1.3050000000000002': [{'creatinine_apache <= 0.575': [{'height <= 165.05': [1, {'creatinine_apache <= 0.535': [{'height <= 166.35': [0, 1]}, 0]}]}, {'weight <= 65.85': [0, {'hematocrit_apache <= 27.9': [{'gcs_verbal_apache <= 1.5': [1, 0]}, 0]}]}]}, {'wbc_apache <= 32.9': [{'creatinine_apache <= 7.77': [{'wbc_apache <= 7.25': [{'hematocrit_apache <= 23.95': [1, 0]}, 1]}, 0]}, 0]}]}
{'bmi <= 22.758655410000003': [{'bun_apache

{'glucose_apache <= 177.5': [{'resprate_apache <= 13.5': [{'age <= 39.0': [0, {'glucose_apache <= 155.5': [{'apache_4a_icu_death_prob <= 0.005': [1, 0]}, {'bun_apache <= 11.5': [0, 1]}]}]}, {'glucose_apache <= 82.5': [{'height <= 176.65': [{'gcs_eyes_apache <= 1.5': [0, 1]}, 1]}, {'bun_apache <= 22.5': [0, {'glucose_apache <= 160.5': [0, 1]}]}]}]}, {'resprate_apache <= 35.5': [{'glucose_apache <= 539.5': [1, 0]}, {'age <= 81.0': [{'glucose_apache <= 395.0': [{'temp_apache <= 35.650000000000006': [0, 1]}, 1]}, 1]}]}]}
{'glucose_apache <= 168.5': [{'resprate_apache <= 15.5': [{'hematocrit_apache <= 41.7': [{'wbc_apache <= 15.15': [1, {'hematocrit_apache <= 31.75': [1, 0]}]}, {'glucose_apache <= 155.0': [0, 1]}]}, {'glucose_apache <= 86.5': [{'wbc_apache <= 5.25': [0, {'heart_rate_apache <= 129.5': [1, 0]}]}, 0]}]}, {'resprate_apache <= 15.5': [{'sodium_apache <= 146.5': [{'height <= 175.25': [1, {'glucose_apache <= 204.5': [0, 1]}]}, 0]}, {'glucose_apache <= 298.5': [{'gcs_motor_apache <

...

{'glucose_apache <= 164.5': [{'resprate_apache <= 13.5': [{'age <= 40.0': [0, {'age <= 83.5': [{'resprate_apache <= 8.5': [0, 1]}, 0]}]}, {'glucose_apache <= 72.5': [{'glucose_apache <= 39.5': [0, {'age <= 57.0': [0, 1]}]}, {'creatinine_apache <= 1.88': [0, {'bmi <= 35.898023124999995': [0, 1]}]}]}]}, {'resprate_apache <= 33.5': [{'creatinine_apache <= 8.73': [{'glucose_apache <= 186.0': [{'bmi <= 27.366713635': [0, 1]}, {'resprate_apache <= 4.5': [0, 1]}]}, 0]}, {'creatinine_apache <= 2.235': [{'creatinine_apache <= 1.9849999999999999': [1, 0]}, 1]}]}]}
{'glucose_apache <= 175.5': [{'age <= 45.5': [{'glucose_apache <= 43.5': [1, {'glucose_apache <= 164.0': [0, 1]}]}, {'bmi <= 34.567565470000005': [{'hematocrit_apache <= 28.45': [{'gcs_verbal_apache <= 4.5': [1, 0]}, 0]}, {'heart_rate_apache <= 54.0': [0, 1]}]}]}, {'glucose_apache <= 257.0': [{'hematocrit_apache <= 35.25': [{'hematocrit_apache <= 28.6': [{'wbc_apache <= 6.949999999999999': [1, 0]}, {'heart_rate_apache <= 143.5': [1, 0]

{'glucose_apache <= 170.5': [{'bmi <= 24.27710136': [{'apache_4a_hospital_death_prob <= 0.045': [0, {'weight <= 51.05': [1, 0]}]}, {'glucose_apache <= 75.5': [{'apache_4a_hospital_death_prob <= 0.07500000000000001': [1, {'hematocrit_apache <= 26.6': [1, 0]}]}, 0]}]}, {'wbc_apache <= 11.3': [{'glucose_apache <= 205.5': [{'wbc_apache <= 6.915': [{'weight <= 162.14999999999998': [1, 0]}, {'map_apache <= 106.5': [1, 0]}]}, {'bun_apache <= 38.5': [1, {'hematocrit_apache <= 29.25': [1, 0]}]}]}, {'glucose_apache <= 232.5': [{'bmi <= 30.842678605000003': [0, 1]}, {'map_apache <= 42.0': [0, 1]}]}]}]}
{'glucose_apache <= 187.0': [{'age <= 56.5': [{'age <= 40.5': [0, {'age <= 50.5': [{'apache_4a_hospital_death_prob <= 0.005': [1, 0]}, {'bun_apache <= 6.5': [1, 0]}]}]}, {'gcs_verbal_apache <= 2.5': [{'apache_4a_hospital_death_prob <= 0.195': [{'wbc_apache <= 5.95': [0, 1]}, {'wbc_apache <= 7.75': [1, 0]}]}, {'glucose_apache <= 82.0': [{'bmi <= 20.1279084': [0, 1]}, {'wbc_apache <= 19.65': [0, 1]}]

{'glucose_apache <= 155.5': [{'gcs_verbal_apache <= 1.5': [{'age <= 42.0': [0, {'glucose_apache <= 107.0': [{'gcs_motor_apache <= 4.5': [0, 1]}, {'gcs_motor_apache <= 2.0': [1, 0]}]}]}, {'resprate_apache <= 28.5': [{'glucose_apache <= 75.5': [1, 0]}, {'age <= 52.5': [0, {'bmi <= 51.451691825': [0, 1]}]}]}]}, {'resprate_apache <= 29.5': [{'glucose_apache <= 185.5': [{'sodium_apache <= 138.5': [{'bmi <= 21.65842455': [0, 1]}, {'age <= 64.5': [1, 0]}]}, 1]}, {'glucose_apache <= 246.5': [{'temp_apache <= 36.95': [0, {'temp_apache <= 39.650000000000006': [1, 0]}]}, {'apache_4a_hospital_death_prob <= 0.87': [{'heart_rate_apache <= 162.5': [1, 0]}, 0]}]}]}]}
{'glucose_apache <= 169.5': [{'age <= 62.5': [{'height <= 158.75': [{'apache_4a_hospital_death_prob <= 0.035': [0, 1]}, 0]}, {'glucose_apache <= 90.5': [{'height <= 158.75': [0, 1]}, {'height <= 175.1': [{'glucose_apache <= 151.0': [0, 1]}, 0]}]}]}, {'glucose_apache <= 210.0': [{'age <= 39.5': [0, {'gcs_eyes_apache <= 2.5': [1, {'temp_apa

{'resprate_apache <= 14.5': [{'bun_apache <= 12.5': [{'creatinine_apache <= 0.575': [1, {'age <= 50.5': [0, {'resprate_apache <= 5.5': [1, 0]}]}]}, {'creatinine_apache <= 0.975': [{'map_apache <= 61.5': [{'gcs_eyes_apache <= 3.5': [1, 0]}, 1]}, {'creatinine_apache <= 10.809999999999999': [1, 0]}]}]}, {'bun_apache <= 20.5': [{'age <= 62.5': [{'bun_apache <= 6.5': [1, 0]}, {'weight <= 105.65': [{'wbc_apache <= 9.5': [0, 1]}, 1]}]}, {'age <= 81.5': [{'age <= 35.0': [1, {'age <= 42.5': [0, 1]}]}, {'map_apache <= 149.5': [{'weight <= 46.9': [1, 0]}, 1]}]}]}]}
{'glucose_apache <= 166.5': [{'age <= 60.5': [{'hematocrit_apache <= 30.45': [{'weight <= 57.2': [0, {'creatinine_apache <= 0.56': [1, 0]}]}, {'weight <= 123.95': [0, 1]}]}, {'bmi <= 30.802607105': [{'temp_apache <= 36.55': [0, {'creatinine_apache <= 3.44': [0, 1]}]}, {'apache_4a_icu_death_prob <= 0.20500000000000002': [{'age <= 66.5': [0, 1]}, 0]}]}]}, {'glucose_apache <= 256.5': [{'apache_4a_icu_death_prob <= 0.435': [{'creatinine_ap

[{'glucose_apache <= 164.5': [{'glucose_apache <= 85.5': [{'hematocrit_apache <= 40.7': [{'resprate_apache <= 15.5': [1,
         {'hematocrit_apache <= 38.349999999999994': [0, 1]}]},
       0]},
     {'creatinine_apache <= 1.9649999999999999': [0,
       {'resprate_apache <= 10.5': [1,
         {'hematocrit_apache <= 24.35': [1, 0]}]}]}]},
   {'resprate_apache <= 30.5': [{'glucose_apache <= 222.5': [{'bmi <= 31.872313034999998': [{'creatinine_apache <= 0.63': [1,
           0]},
         {'glucose_apache <= 219.0': [1, 0]}]},
       {'height <= 191.79500000000002': [{'apache_4a_icu_death_prob <= 0.76': [1,
           0]},
         0]}]},
     {'glucose_apache <= 249.5': [{'bmi <= 52.78616082': [0, 1]},
       {'apache_4a_hospital_death_prob <= 0.865': [1, 0]}]}]}]},
 {'glucose_apache <= 169.5': [{'glucose_apache <= 82.5': [{'bmi <= 27.938894565': [{'wbc_apache <= 34.05': [{'age <= 77.0': [0,
           1]},
         1]},
       {'age <= 52.5': [0, 1]}]},
     {'bmi <= 26.619904575': 

## Step4 : Make predictions with the random forest
> Note: Please print the f1-score of the predictions of each decision tree

In [57]:
def make_prediction_forest(forest, data):
  """
  This function will use the pre-trained random forest to make the predictions
  args:
  * forest: the random forest
  * data: the data used to predict
  return:
  * y_prediction: the predicted results
  """
  
  y_prediction = []
  for i in range(len(forest)):
      y_prediction.append(make_prediction(forest[i], data))
  return y_prediction

In [62]:
y_test = make_prediction_forest(forest, x_validation)
y_pred_test_forest = make_prediction_forest(forest, x_test)
bad_result = []
count_good_tree = 0


for i in range(len(forest)):
  score = calculate_score(y_validation, y_test[i])
  if(score < 0.55):
    bad_result.append(1)
  else:
    bad_result.append(0)
    count_good_tree = count_good_tree + 1
  print(score)
y_test_tree = []
y_pred_test = []
for i in range(len(x_validation)):
  y_test_tree.append(0)
for i in range(len(x_test)):
  y_pred_test.append(0)

for i in range(len(x_validation)):
  for j in range(len(forest)):
    if(bad_result[j] == 0):
      y_test_tree[i] += y_test[j][i]
      
        
        

for i in range(len(x_validation)):
  if(y_test_tree[i] >= count_good_tree/2):
    y_test_tree[i] = 1
  else:
    y_test_tree[i] = 0
  


for i in range(len(x_test)):
  for j in range(len(forest)):
    if(bad_result[j] == 0):
      y_pred_test[i] += y_pred_test_forest[j][i]

for i in range(len(x_test)):
  if(y_pred_test[i] >= count_good_tree/2):
    y_pred_test[i] = 1
  else:
    y_pred_test[i] = 0

ans_f1score1 = calculate_score(y_validation, y_test_tree)
print("ans_f1score = ", ans_f1score1)

0.6106471816283925
0.6706476802939826
0.6407950780880265
0.6619593998234775
0.5963446475195824
0.6568580920740389
0.666358595194085
0.4986904138292301
0.6148409893992933
0.6075085324232082
0.6225402504472272
0.6402048655569782
0.5526189332051898
0.4557935200439318
0.6215098241985522
0.6025738077214232
0.5658986175115207
0.5993820803295571
0.6878402903811253
0.610917537746806
0.6157697121401752
0.6584524396254312
0.5240750126710593
0.5676190476190476
0.6633348477964561
0.6844074844074844
0.570907432740538
0.5530227948463826
0.5691837625491052
0.6548837209302326
0.4337748344370861
0.47281795511221947
0.6143604374702807
0.6490486257928119
0.6352715040845748
0.542830540037244
0.5796831314072693
0.5717684377478192
0.646341463414634
0.5854020007410152
0.6924660354055167
0.612184249628529
0.6802250108178277
0.5272727272727273
0.64908359409924
0.492639842983317
0.6239526860522425
0.48872950819672134
0.6322820037105752
0.390625
0.684373616644533
0.6851363236587512
0.6414565826330533
0.585737179

0.6728025770823746
0.6216216216216216
0.6083418107833163
0.6591422121896162
0.4467504096122337
0.6912405866032502
0.5502063273727648
0.62032598274209
0.573502722323049
0.496630378434422
0.6128868594622019
0.6332196785192401
0.5491051942383239
0.6201005025125628
0.5895390070921985
0.6548107615139078
0.6610455311973018
0.6712328767123288
0.6972010178117048
0.6515291936978685
0.6770293609671848
0.540590405904059
0.5756457564575646
0.6362735381565906
0.6518452645620275
0.6103092783505154
0.6097201767304861
0.41652123327515994
0.27951153324287653
0.6533575317604355
0.5326781326781328
0.549172346640701
0.6540240518038853
0.6513157894736842
0.6173608407940833
0.6550179211469533
0.6374464030490711
0.501852832186342
0.5154943433349729
0.607707509881423
0.5785938199731303
0.6480965645311049
0.5796976241900648
0.6607476635514018
0.3794326241134752
0.5332040717401842
0.6588520614389652
ans_f1score =  0.7165824529168581


## Step5 : Write the Output File
Save your predictions from the **random forest** in a csv file, named as **hw2_advanced.csv**

In [59]:
advanced = []

for i in range(len(y_pred_test)):
  advanced.append(y_pred_test[i])

In [60]:
advanced_path = 'hw2_advanced.csv'
pd.DataFrame(advanced).to_csv(advanced_path, header = None, index = None)